# Objective Binge Classifier #

Import dependencies

In [ ]:
!pip install openpyxl
import pandas as pd

Set paths and columns of response document

In [ ]:
path_in = '../resources/data_ignored/report.merged-vj-unfiltered.xlsx'
df = pd.read_excel(path_in)

col_name_post_instance_key = 'POST_instance_key'
col_name_post_loc = 'POST_LOC' # Self-reported Loss Of Control
col_name_pre_type = 'PRE_TYPE'
col_post_sub_binge = 'POST_SUB_BINGE'

loc_true = 1 # Self-reported Loss Of Control required value
sub_binge_true = 1 # Subjective Binge required value
pre_type_category_other = ['Anders']

columns_type_counts = df[col_name_pre_type].value_counts()
print('counts', columns_type_counts)

Set paths and columns of config document

In [ ]:
path_config = '../resources/config/binge_config.xlsx'

df_config = pd.read_excel(path_config)
print(df_config.columns)

Set paths and columns for result document

In [ ]:
column_objective_binge = 'objective_binge'            
path_out = '../resources/data_ignored/report.merged-vj-binge.xlsx'

Classification methods

In [ ]:
def isNaN(num):
    return num != num

def get_binge_value(row, category_number):
    if row[col_name_pre_type] in pre_type_category_other:
        col_name_bing_int = 'other_int'
    else:
        col_name_bing_int = 'meals_int'
    return df_config.loc[df_config['#'] == category_number, col_name_bing_int].item()

def objective_binge(row):
    if isNaN(row[col_name_post_instance_key]):
        return "IGNORE_POST_INSTANCE_MISSING"
    if row[col_name_post_loc] != loc_true:
        return "IGNORE_POST_LOC_FALSE"
    if row[col_post_sub_binge] != sub_binge_true:
        return "IGNORE_POST_SUB_BINGE_FALSE"
    for x in range(24):
        category_number = x + 1
        col_name_post_intake_por = 'POST_INTAKE_POR_'+str(category_number)
        if not isNaN(row[col_name_post_intake_por]):
            value = row[col_name_post_intake_por]
            binge_value = get_binge_value(row, category_number)
            if value > binge_value:
                return True
            else:
                return False
    return "NO_PORTION"


Preform classification and write results

In [ ]:
df[column_objective_binge] = df.apply(objective_binge, axis=1)

column_objective_binge_counts = df[column_objective_binge].value_counts()
print('total PRE entries', len(df.index))
print(column_objective_binge) 
print(column_objective_binge_counts)

print('writing result')
df.to_excel(path_out)
print('done!')